# Hands on examples from 
[[MlMastery Arcticle on the LSTM TS Subject]](https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/#:~:text=Multivariate%20time%20series%20data%20means%20data%20where%20there%20is%20more)

## Univariate CNN-LSTM 

In [24]:
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

In [25]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

In [26]:
# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
# choose a number of time steps
n_steps = 4
# split into samples
X, y = split_sequence(raw_seq, n_steps)

In [27]:
X.shape, y.shape

((5, 4), (5,))

In [28]:
X[:2]

array([[10, 20, 30, 40],
       [20, 30, 40, 50]])

In [29]:
# reshape from [samples, timesteps] into [samples, subsequences, timesteps, features]
n_features = 1
n_seq = 2
n_steps = 2

X = X.reshape((X.shape[0], n_seq, n_steps, n_features))

In [30]:
X.shape, y.shape

((5, 2, 2, 1), (5,))

In [31]:
X[:2]

array([[[[10],
         [20]],

        [[30],
         [40]]],


       [[[20],
         [30]],

        [[40],
         [50]]]])

In [61]:
# define model
model = Sequential()
model.add(TimeDistributed(Conv1D(filters=64, kernel_size=1, activation='relu'), input_shape=(None, n_steps, n_features)))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [62]:
%%time
# fit model
model.fit(X, y, epochs=500, verbose=0)
# demonstrate prediction
x_input = array([60, 70, 80, 90])
x_input = x_input.reshape((1, n_seq, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[100.86332]]
CPU times: user 9.6 s, sys: 5.2 s, total: 14.8 s
Wall time: 5.34 s


## Example for Multivariate LSTM Model

In [1]:
import numpy as np

In [2]:
in_seq1 = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = np.array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = np.array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

In [3]:
in_seq1.shape

(9,)

In [4]:
in_seq1

array([10, 20, 30, 40, 50, 60, 70, 80, 90])

In [5]:
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = np.hstack((in_seq1, in_seq2, out_seq))

In [6]:
in_seq1.shape

(9, 1)

In [7]:
in_seq1

array([[10],
       [20],
       [30],
       [40],
       [50],
       [60],
       [70],
       [80],
       [90]])

In [8]:
dataset

array([[ 10,  15,  25],
       [ 20,  25,  45],
       [ 30,  35,  65],
       [ 40,  45,  85],
       [ 50,  55, 105],
       [ 60,  65, 125],
       [ 70,  75, 145],
       [ 80,  85, 165],
       [ 90,  95, 185]])

In [9]:
# split Fn
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [10]:
# choose a number of time steps
n_steps = 3
# convert into input/output
X, y = split_sequences(dataset, n_steps)
print(X.shape, y.shape)

(7, 3, 2) (7,)


In [11]:
X[0], y[0]

(array([[10, 15],
        [20, 25],
        [30, 35]]),
 65)

In [12]:
# first three elements
for i in range(3):
    print(X[i], y[i])

[[10 15]
 [20 25]
 [30 35]] 65
[[20 25]
 [30 35]
 [40 45]] 85
[[30 35]
 [40 45]
 [50 55]] 105


## Using LSTM on the data

In [13]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Flatten

In [14]:
# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]

# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 50)                10600     
_________________________________________________________________
dense (Dense)                (None, 1)                 51        
Total params: 10,651
Trainable params: 10,651
Non-trainable params: 0
_________________________________________________________________


In [16]:
# fit model
model.fit(X, y, epochs=200, verbose=0)

In [22]:
# demonstrate prediction
x_input = np.array([[80, 85], [90, 95], [100, 105]])
x_input_r = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input_r, verbose=0)
print(yhat)

[[206.99915]]


In [20]:
n_steps, n_features

(3, 2)

In [23]:
x_input, x_input.shape

(array([[ 80,  85],
        [ 90,  95],
        [100, 105]]),
 (3, 2))

In [24]:
x_input_r, x_input_r.shape

(array([[[ 80,  85],
         [ 90,  95],
         [100, 105]]]),
 (1, 3, 2))